In [111]:
import ROOT
import numpy as np
import time
import math
import matplotlib.pyplot as plt

### Constants

In [112]:
nr_of_channels = int(3.6 * 1e6)
channels_per_module = int(15*30)
total_modules = int(C_total_channels/C_channels_per_module)

### Noise Generator Class

In [113]:
class noise_generator:
    def __init__(self, threshold, channel_count): # channel_count = C_total_channels
        global global_counter
        low = 0
        high = 1000
        noise = ROOT.TF1("noise_gauss", "TMath::Gaus(x, 0, 5)", low, high)
        
        #integrate normal dist. to get exact prob. above threshold and multiply with channel nr to get avg.
        self._mean = channel_count * noise.Integral(threshold, high)/noise.Integral(low, high)/2 
        #self._prng = ROOT.TRandom3(time.time_ns())
        global_counter += 1
        self._prng = ROOT.TRandom3(int(time.time()) + global_counter)

    def generate(self):
        """The first return value is the exact number of noise activated channels in the small window and
            the second return value is the estimated number of noise activated channels in the small window. (estimated from large window)
            (both results are in small window, both results are noise above threshold)
            noise_exact, noise_est = noise.generate()"""
        # Large time window 25ns
        # Small time window 3.125ns
        prng = self._prng
        total = prng.Poisson(self._mean) # number of noise above threshold in large window in this run.
        narrow_exact = 0
        if total <= 5:
            for i in range(total):
                if prng.Uniform(25) < 3.125:
                    narrow_exact += 1
        else:
            # faster to use numpy
            #narrow_exact = (np.random.uniform(low=0, high=25, size=total) < 3.125).sum()
            narrow_exact = (np.random.rand(total) < 0.125).sum()  # 3.125/25 = 0.125
        
        noise_per_time = (total - narrow_exact)/(25 - 3.125)
        estimate = 3.125 * noise_per_time
        return narrow_exact, estimate




 ### Signal Generator Class
 


In [114]:
class signal_generator:
    def __init__(self, threshold, real_expected_voltage, guess_expected_voltage): # expected_voltage = expected peak voltage  (mu)
        global global_counter
        low = -100
        high = 1000
        self._threshold = threshold
        self._expected_voltage = real_expected_voltage
        signal_func = ROOT.TF1("sth", "TMath::Landau(x, %f, %f)" % (guess_expected_voltage, guess_expected_voltage/4), low, high)
        self.fraction_above = signal_func.Integral(threshold, high)/signal_func.Integral(low, high)
        #self._prng = ROOT.TRandom3(time.time_ns())
        global_counter += 1
        self._prng = ROOT.TRandom3(int(time.time()) + global_counter)
    
    def generate(self, expected_count):
        """The paramater is the expected number of particles that reach the detector.
            The first return value is the number of particles that reached the detector and
            the second return value is the number of activated channels.
            signal_real_total, signal_above = signal.generate(30)"""
        prng = self._prng
        expected_voltage = self._expected_voltage
        threshold = self._threshold
        
        signal_count = prng.Poisson(expected_count)
        above_threshold_count = 0
        sigma = expected_voltage/4
        for i in range(signal_count):
            if prng.Landau(expected_voltage, sigma) > threshold: # 1/4 is just the refernce 25/100
                above_threshold_count += 1
        return signal_count, above_threshold_count
    

## Simulate noise and signal on detector

In [170]:
threshold = 10
noise = noise_generator(threshold, nr_of_channels)
noise_exact, noise_estimate = noise.generate()
print("Exact number of noise activated channels in the small window: %d" %noise_exact)
print ("Estimated number of noise activated channels in the small window: %d" %noise_estimate)

exected_count = 30 #expected number of particles reaching the detector 
mean_peak_V = 100
guess_peak_V = 100
signal = signal_generator (threshold, mean_peak_V, guess_peak_V)
signal_real_total, signal_above_threshold = signal.generate(exected_count)
print("Number of particles that reach the detector: %d" %signal_real_total)
print("Number of signal-activated channels: %d" %signal_above_threshold)

Exact number of noise activated channels in the small window: 10139
Estimated number of noise activated channels in the small window: 10254
Number of particles that reach the detector: 22
Number of signal-activated channels: 22
